# DataJoint U24 - Workflow DeepLabCut

Change into the parent directory to find the `dj_local_conf.json` file. 

In [1]:
import os
# change to the upper level folder to detect dj_local_conf.json
if os.path.basename(os.getcwd())=='notebooks': os.chdir('..')

## Write NWB file

In [37]:
from workflow_deeplabcut.pipeline import model, lab, session
from workflow_deeplabcut.export import session_to_nwb, dlc_session_to_nwb


session_key = (session.Session).fetch(limit=1)[0]
pose_key = (model.PoseEstimation).fetch("KEY")[0]
session_kwargs = dict(
    lab_key=(lab.Lab & "lab='LabA'").fetch1(),
    project_key=(lab.Protocol & "protocol='ProtA'").fetch1(),
    protocol_key=(lab.Project & "project='ProjA'").fetch1(),
)

In [39]:
session_kwargs

{'lab_key': {'lab': 'LabA',
  'lab_name': 'The Example Lab',
  'institution': 'Example Uni',
  'address': "'221B Baker St",
  'time_zone': 'London NW1 6XE'},
 'project_key': {'protocol': 'ProtA',
  'protocol_type': 'IRB expedited review',
  'protocol_description': 'Protocol for managing data ingestion'},
 'protocol_key': {'project': 'ProjA',
  'project_description': 'Example project to populate element-lab'}}

In [40]:
session_to_nwb(session_key, session_kwargs)

root pynwb.file.NWBFile at 0x140380074352352
Fields:
  file_create_date: [datetime.datetime(2022, 7, 27, 16, 13, 20, 272132, tzinfo=tzlocal())]
  identifier: a529662a-ba2b-4cff-b399-eadbee18cd67
  session_description: Model Training
  session_id: subject6_2021-06-01T13:33:33
  session_start_time: 2021-06-01 17:33:33+00:00
  subject: subject pynwb.file.Subject at 0x140379806760144
Fields:
  subject_id: subject6

  timestamps_reference_time: 2021-06-01 17:33:33+00:00

In [ ]:
fpath = dlc_session_to_nwb(pose_key,
                           use_element_session=True, 
                           session_kwargs=session_kwargs)

In [ ]:
# Needs NWB widget for exploring

## Bugfix work

In [ ]:
import numpy as np
from dlc2nwb.utils import get_movie_timestamps 
from element_interface.utils import find_full_path
from workflow_deeplabcut.paths import get_dlc_root_data_dir
vid_fp = str(find_full_path(get_dlc_root_data_dir(), 'from_top_tracking/videos/test.mp4'))
timestamps = get_movie_timestamps(str(vid_fp))

: 

In [1]:
from pathlib import Path
import os; os.chdir('..')
from workflow_deeplabcut.pipeline import model, lab, session
from workflow_deeplabcut.export import dlc_session_to_nwb

session_key = (session.Session).fetch(limit=1)[0]
pose_key = (model.PoseEstimation).fetch("KEY")[0]
key = pose_key
write_file = True
subject_id = key["subject"]
output_dir = model.PoseEstimationTask.infer_output_dir(key)
config_file = str(output_dir / "dj_dlc_config.yaml")
video_name = Path((model.VideoRecording.File & key).fetch1("file_path")).stem
h5file = str(list(output_dir.glob(f"{video_name}*h5"))[0])

[2022-07-27 17:04:20,055][INFO]: Connecting cbroz@dss-db.datajoint.io:3306
[2022-07-27 17:04:20,118][INFO]: Connected cbroz@dss-db.datajoint.io:3306


Loading DLC 2.2.1.1...


/Users/cb/miniconda3/envs/nwb/lib/python3.9/site-packages/statsmodels/compat/pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex
/Users/cb/miniconda3/envs/nwb/lib/python3.9/site-packages/deeplabcut/__init__.py:81: UserWarning: 
        As PyTorch is not installed, unsupervised identity learning will not be available.
        Please run `pip install torch`, or ignore this warning.
        
  warnings.warn(


DLC loaded in light mode; you cannot use any GUI (labeling, relabeling and standalone GUI)


In [2]:
import os
import pandas as pd
from dlc2nwb import utils


FILEPATH = str(
    find_full_path(
        get_dlc_root_data_dir(), 
        (
            'from_top_tracking/videos/device_Camera1_recording_1_model_FromTop-latest/'
            +'test-2sDLC_mobnet_100_from_top_trackingFeb23shuffle1_103000_ind1.h5'
        )
        )
)
CONFIGPATH = str(
    find_full_path(get_dlc_root_data_dir(), 'from_top_tracking/config.yaml')
)


def test_round_trip_conversion():
    df_ref = pd.read_hdf(FILEPATH)
    nwbfile = utils.convert_h5_to_nwb(
        CONFIGPATH,
        FILEPATH,
    )[0]
    df = utils.convert_nwb_to_h5(nwbfile).droplevel("individuals", axis=1)
    
    pd.testing.assert_frame_equal(df[:-3], df_ref[:-3])


def test_multi_animal_round_trip_conversion(tmp_path):
    dfs = []
    n_animals = 3
    for i in range(1, n_animals + 1):
        temp = utils._ensure_individuals_in_header(
            pd.read_hdf(FILEPATH),
            f"animal_{i}",
        )
        dfs.append(temp)
    df = pd.concat(dfs, axis=1)
    path_fake_df = str(tmp_path / os.path.split(FILEPATH)[1])
    df.to_hdf(path_fake_df, key="fake")
    nwbfiles = utils.convert_h5_to_nwb(CONFIGPATH, path_fake_df)
    assert len(nwbfiles) == n_animals

In [3]:
test_round_trip_conversion()

AssertionError: DataFrame.index are different

DataFrame.index values are different (99.16667 %)
[left]:  Int64Index([0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
            ...
            1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
           dtype='int64', length=120)
[right]: Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
            ...
            110, 111, 112, 113, 114, 115, 116, 117, 118, 119],
           dtype='int64', length=120)

In [73]:
df[:-3]

scorer    DLC_mobnet_100_from_top_trackingFeb23shuffle1_103000              \
bodyparts                                                 head               
coords                                                       x           y   
0                                                 241.037445    316.333527   
0                                                 239.048523    319.177246   
0                                                 240.238083    320.525360   
0                                                 238.537003    322.323914   
0                                                 237.968628    324.072052   
..                                                       ...           ...   
1                                                 263.530640    370.027618   
1                                                 265.803284    370.275970   
1                                                 269.777344    371.322601   
1                                                 270.854828    371.892914   
1                                                 271.488251    373.100311   

scorer                                                               \
bodyparts             bodycenter                           tailbase   
coords    likelihood           x           y likelihood           x   
0           0.999850  246.781891  298.727905   0.999998  256.201874   
0           0.999905  246.216980  299.357452   0.999997  255.819000   
0           0.999899  244.459229  301.309052   0.999999  255.704605   
0           0.999941  242.014679  302.864990   0.999999  254.423828   
0           0.999941  240.899963  303.458252   0.999998  252.180069   
..               ...         ...         ...        ...         ...   
1           0.999951  243.028030  363.364655   0.999972  231.534760   
1           0.999907  244.662079  363.382416   0.999983  232.552444   
1           0.999931  246.925781  363.787506   0.999982  233.119934   
1           0.999961  248.681427  364.710205   0.999965  234.898987   
1           0.999991  250.325821  366.870453   0.999972  235.643646   

scorer                            
bodyparts                         
coords              y likelihood  
0          278.553345   0.999998  
0          280.199982   0.999996  
0          280.938965   0.999995  
0          282.015533   0.999990  
0          280.898529   0.999977  
..                ...        ...  
1          348.766998   0.999997  
1          350.001251   0.999997  
1          352.345856   0.999997  
1          356.035370   0.999996  
1          356.813232   0.999989  

[120 rows x 9 columns]

In [70]:
df_ref[:-3]

scorer    DLC_mobnet_100_from_top_trackingFeb23shuffle1_103000              \
bodyparts                                                 head               
coords                                                       x           y   
0                                                 241.037445    316.333527   
1                                                 239.048523    319.177246   
2                                                 240.238083    320.525360   
3                                                 238.537003    322.323914   
4                                                 237.968628    324.072052   
..                                                       ...           ...   
115                                               263.530640    370.027618   
116                                               265.803284    370.275970   
117                                               269.777344    371.322601   
118                                               270.854828    371.892914   
119                                               271.488251    373.100311   

scorer                                                               \
bodyparts             bodycenter                           tailbase   
coords    likelihood           x           y likelihood           x   
0           0.999850  246.781891  298.727905   0.999998  256.201874   
1           0.999905  246.216980  299.357452   0.999997  255.819000   
2           0.999899  244.459229  301.309052   0.999999  255.704605   
3           0.999941  242.014679  302.864990   0.999999  254.423828   
4           0.999941  240.899963  303.458252   0.999998  252.180069   
..               ...         ...         ...        ...         ...   
115         0.999951  243.028030  363.364655   0.999972  231.534760   
116         0.999907  244.662079  363.382416   0.999983  232.552444   
117         0.999931  246.925781  363.787506   0.999982  233.119934   
118         0.999961  248.681427  364.710205   0.999965  234.898987   
119         0.999991  250.325821  366.870453   0.999972  235.643646   

scorer                            
bodyparts                         
coords              y likelihood  
0          278.553345   0.999998  
1          280.199982   0.999996  
2          280.938965   0.999995  
3          282.015533   0.999990  
4          280.898529   0.999977  
..                ...        ...  
115        348.766998   0.999997  
116        350.001251   0.999997  
117        352.345856   0.999997  
118        356.035370   0.999996  
119        356.813232   0.999989  

[120 rows x 9 columns]